# AI-PSCI-015: Molecular Docking with AutoDock Vina
## 📘 SOLUTION NOTEBOOK

**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 9 | Module: Prototyping | Estimated Time: 90-120 minutes**

**Prerequisites**: AI-PSCI-001 through AI-PSCI-014

---

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. Prepare protein and ligand structures for molecular docking
2. Define a binding site and configure docking parameters
3. Run AutoDock Vina and interpret docking scores
4. Visualize docking poses and compare to experimental structures
5. Understand the strengths and limitations of physics-based docking

---

## 📚 Background

### What is Molecular Docking?

**Molecular docking** is a computational technique that predicts how a small molecule (ligand) binds to a protein target. It answers two key questions:

1. **Where** does the ligand bind? (binding pose/geometry)
2. **How strongly** does it bind? (binding affinity/score)

Docking is central to drug discovery because it allows researchers to:
- Screen millions of compounds *in silico* before synthesis
- Understand drug-target interactions at atomic detail
- Design improved drugs by optimizing binding interactions
- Predict effects of protein mutations on drug binding

### How AutoDock Vina Works

AutoDock Vina uses a combination of:

1. **Sampling algorithm**: Iterated local search + gradient optimization to explore ligand conformations
2. **Scoring function**: Empirical function combining:
   - Steric interactions (shape complementarity)
   - Hydrophobic effects
   - Hydrogen bonding
   - Torsional penalties

The output is a **docking score** in kcal/mol (more negative = stronger predicted binding).

### Limitations to Keep in Mind

- **Protein flexibility**: Vina treats the protein as rigid (though you can make specific residues flexible)
- **Scoring accuracy**: Scores correlate with affinity but are not exact binding energies
- **Binding site required**: You must define where to dock (unlike AI methods like DiffDock)
- **Water and ions**: Often ignored, though they can be critical for binding

### Key Concepts

- **PDBQT format**: Special file format that includes partial charges and atom types
- **Binding box**: The 3D region where docking is performed
- **Exhaustiveness**: Number of independent docking runs (higher = more thorough but slower)
- **RMSD**: Root Mean Square Deviation - measures how similar two poses are

---

## 🛠️ Setup

Run this cell to install required packages:

In [32]:
#@title 🛠️ Install required packages (click ▶ to run)
!pip install vina -q
!pip install gemmi -q
!pip install meeko -q
!pip install rdkit -q
!pip install py3Dmol -q
!pip install biopython -q
!pip install scipy -q
!pip install openbabel-wheel -q  # For PDB -> PDBQT conversion
print("✅ All packages installed!")

✅ All packages installed!


Import required libraries:

In [33]:
# Core libraries
import os
import requests
import numpy as np
import pandas as pd
from pathlib import Path

# Molecular handling
from rdkit import Chem
from rdkit.Chem import AllChem, Draw
from meeko import MoleculePreparation, PDBQTMolecule
from meeko import PDBQTWriterLegacy

# Docking
from vina import Vina

# Visualization
import py3Dmol
import matplotlib.pyplot as plt

# Structure analysis
from Bio.PDB import PDBParser, PDBIO, Select
from scipy.spatial.distance import cdist

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


---

## 🎯 Target Configuration

Select your drug target. This selection will be used throughout this talktorial.

In [ ]:
#@title 🎯 Select Your Drug Target
TARGET = "DHFR" #@param ["DHFR", "ABL1", "EGFR", "AChE", "COX-2", "DPP-4"]

# Complete target configuration with all identifiers
# Note: For docking, we use PDB structures that have the reference drug bound
# This allows RMSD validation of docking poses
TARGET_CONFIG = {
    "DHFR": {
        "pdb": "2W9S",  # E. coli DHFR with trimethoprim + NADPH
        "uniprot": "P0ABQ4",
        "chembl": "CHEMBL202",
        "drug": "Trimethoprim",
        "drug_smiles": "COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC",
        "ligand_code": "TOP",  # Trimethoprim ligand code in 2W9S
        "description": "Dihydrofolate reductase - antibiotic target"
    },
    "ABL1": {
        "pdb": "1IEP",
        "uniprot": "P00519",
        "chembl": "CHEMBL1862",
        "drug": "Imatinib",
        "drug_smiles": "Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1",
        "ligand_code": "STI",
        "description": "Tyrosine kinase - cancer target (CML)"
    },
    "EGFR": {
        "pdb": "1M17",
        "uniprot": "P00533",
        "chembl": "CHEMBL203",
        "drug": "Erlotinib",
        "drug_smiles": "COCCOc1cc2ncnc(Nc3cccc(C#C)c3)c2cc1OCCOC",
        "ligand_code": "AQ4",
        "description": "Receptor tyrosine kinase - cancer target (NSCLC)"
    },
    "AChE": {
        "pdb": "4EY7",
        "uniprot": "P22303",
        "chembl": "CHEMBL220",
        "drug": "Donepezil",
        "drug_smiles": "COc1cc2CC(CC2cc1OC)CN1CCc2ccccc2C1=O",
        "ligand_code": "E20",
        "description": "Acetylcholinesterase - Alzheimer's target"
    },
    "COX-2": {
        "pdb": "3LN1",
        "uniprot": "P35354",
        "chembl": "CHEMBL230",
        "drug": "Celecoxib",
        "drug_smiles": "Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2)cc1",
        "ligand_code": "CEL",
        "description": "Cyclooxygenase-2 - inflammation target"
    },
    "DPP-4": {
        "pdb": "1X70",
        "uniprot": "P27487",
        "chembl": "CHEMBL284",
        "drug": "Sitagliptin",
        "drug_smiles": "Fc1cc(F)c(C[C@H](N)CC(=O)N2CCn3c(nnc3C(F)(F)F)C2)c(F)c1F",
        "ligand_code": "715",
        "description": "Dipeptidyl peptidase-4 - diabetes target"
    }
}

# Get configuration for selected target
config = TARGET_CONFIG[TARGET]

print(f"✅ Target: {TARGET}")
print(f"   Description: {config['description']}")
print(f"   PDB: {config['pdb']} | UniProt: {config['uniprot']}")
print(f"   Reference Drug: {config['drug']}")
print(f"   Ligand Code in PDB: {config['ligand_code']}")

---

## 🔬 Guided Inquiry 1: Downloading and Preparing the Protein

### Context

Before docking, we need to prepare the protein structure. This involves:
1. Downloading the PDB structure
2. Extracting only the protein atoms (removing water, ions, existing ligands)
3. Adding hydrogen atoms
4. Converting to PDBQT format (required by Vina)

The PDBQT format is similar to PDB but includes **partial charges (Q)** and **atom types (T)** needed by the docking scoring function.

### Your Task

Using your AI assistant, write code to:
1. Download the PDB structure for your target
2. Extract the protein atoms (chain A) and save to a clean PDB file
3. Also extract the ligand for later binding site definition
4. Display the protein structure with py3Dmol

💡 **Prompting Tips**:
- Ask: "How do I download a PDB file from RCSB and extract only protein atoms?"
- Use BioPython's PDBParser and PDBIO for structure manipulation
- Remember to handle multi-chain structures appropriately

### Verification

After running your code, confirm:
- [ ] PDB file downloaded successfully
- [ ] Protein PDB contains only protein atoms (no HETATM)
- [ ] 3D visualization displays the protein

📓 **Lab Notebook**: Document the number of residues in your protein structure.

In [ ]:
# Your code here



### 📝 Solution Notes

**Key concept**: PDB files often contain more than just the protein - water molecules, ions, ligands, and multiple chains. For docking, we need a clean protein structure.

**Why Chain A**: Many crystal structures have multiple identical chains. We use Chain A to reduce file size and computational time.

**Hetfield in BioPython**: Residues have an ID tuple (hetfield, resseq, icode). Regular amino acids have blank hetfield (' '), while water is 'W' and ligands are 'H_XXX'.

**Common issues**:
- Some targets have ligands with different codes than expected
- Multi-chain complexes may need different chain selection

---

## 🔬 Guided Inquiry 2: Preparing the Ligand

### Context

Now we need to prepare our reference drug for docking. This involves:
1. Creating a 3D conformation from SMILES
2. Adding hydrogen atoms
3. Computing partial charges
4. Converting to PDBQT format using Meeko

**Meeko** is a modern Python library for preparing ligands for AutoDock. It handles charge calculation and PDBQT conversion automatically.

### Your Task

Using your AI assistant, write code to:
1. Create an RDKit molecule from the drug's SMILES
2. Generate a 3D conformation with hydrogens
3. Use Meeko to prepare the molecule for docking
4. Save as PDBQT file

💡 **Prompting Tips**:
- Ask: "How do I convert a SMILES string to PDBQT format using RDKit and Meeko?"
- Meeko's MoleculePreparation handles most of the complexity
- Make sure to use AllChem.EmbedMolecule for 3D coordinates

### Verification

After running your code, confirm:
- [ ] Molecule created from SMILES
- [ ] 3D conformation generated
- [ ] PDBQT file saved

📓 **Lab Notebook**: Record the number of rotatable bonds in your ligand.

In [ ]:
# Your code here



### 📝 Solution Notes

**Key concept**: PDBQT files contain partial charges (Gasteiger by default) and atom types that Vina's scoring function needs.

**Why Meeko**: It's the modern replacement for older tools like MGLTools. It handles:
- Proper atom typing for AutoDock
- Charge calculation
- Identification of rotatable bonds (for flexible docking)

**Rotatable bonds matter**: More rotatable bonds = more degrees of freedom = slower docking. Typical drug-like molecules have 0-10.

**Common issues**:
- Embedding can fail for complex molecules - use `useRandomCoords=True`
- Stereochemistry must be explicit in SMILES for correct 3D structure

---

## 🔬 Guided Inquiry 3: Defining the Binding Site

### Context

AutoDock Vina requires you to define a **search box** - the 3D region where docking will be performed. This is typically centered on the known binding site.

If we have a co-crystallized ligand, we can use its position to define the binding site. Otherwise, we might use:
- Binding site detection algorithms
- Known active site residues from literature
- Predictions from AI tools

### Your Task

Using your AI assistant, write code to:
1. Extract the center coordinates from the crystallographic ligand (if available)
2. Calculate appropriate box dimensions (ligand size + buffer)
3. If no ligand is available, define the box based on known active site residues
4. Visualize the binding box on the protein

💡 **Prompting Tips**:
- Ask: "How do I calculate the geometric center of atoms in a PDB file?"
- A buffer of 5-10 Å around the ligand is typical
- The box size affects docking time - larger boxes are slower

### Verification

After running your code, confirm:
- [ ] Center coordinates calculated
- [ ] Box dimensions are reasonable (typically 15-25 Å per dimension)
- [ ] Visualization shows the box encompasses the binding site

📓 **Lab Notebook**: Record the box center coordinates and dimensions.

In [ ]:
# Your code here



### 📝 Solution Notes

**Key concept**: The binding box determines where Vina will search for poses. Too small = might miss the correct pose. Too large = slower and more false positives.

**Typical box sizes**:
- Small binding site (DHFR): 15-20 Å per dimension
- Medium binding site (kinases): 20-25 Å
- Large/deep binding site (AChE): 25-30 Å

**Buffer rationale**: 5-10 Å buffer ensures:
- Room for different binding orientations
- Captures all relevant protein-ligand interactions
- Accounts for ligand conformational flexibility

**Alternative approaches**:
- Use catalytic residues if known (e.g., DPP-4: Ser630, His740, Asp708)
- Run binding site detection (DoGSiteScorer, fpocket)

---

## 🔬 Guided Inquiry 4: Running AutoDock Vina

### Context

Now we're ready to run the actual docking calculation! Key parameters include:

- **exhaustiveness**: Number of independent runs (default 8, higher = more thorough)
- **n_poses**: Number of top poses to return (typically 5-20)
- **seed**: Random seed for reproducibility

For a typical drug-like molecule, docking takes 1-5 minutes depending on:
- Number of rotatable bonds in the ligand
- Size of the binding box
- Exhaustiveness setting

### Your Task

Using your AI assistant, write code to:
1. Prepare the protein PDBQT (add hydrogens and charges)
2. Configure Vina with the binding box parameters
3. Run the docking calculation
4. Display the docking scores

💡 **Prompting Tips**:
- Ask: "How do I run AutoDock Vina docking using the Python vina library?"
- The vina package provides a Pythonic interface to Vina
- Start with exhaustiveness=8 for reasonable speed

### Verification

After running your code, confirm:
- [ ] Docking completed without errors
- [ ] Multiple poses generated (typically 5-10)
- [ ] Best score is reasonably negative (-5 to -12 kcal/mol for drug-like molecules)

📓 **Lab Notebook**: Record the best docking score and the time taken.

In [ ]:
# Your code here



### 📝 Solution Notes

**Understanding the scores**:
- More negative = stronger predicted binding
- Typical drug-like molecules: -6 to -12 kcal/mol
- Scores below -12 kcal/mol are rare and should be viewed skeptically

**RMSD columns**:
- `l.b.` = lower bound (symmetry-corrected)
- `u.b.` = upper bound (no correction)
- Compared to the best-scoring pose (pose 1)

**Exhaustiveness trade-off**:
- Higher exhaustiveness = more thorough sampling but slower
- For screening: 8-16 is reasonable
- For final poses: 32+ is recommended

**Score interpretation warning**:
- Scores are NOT calibrated binding energies
- Cannot directly compare across different targets
- Best used for ranking compounds within a target

---

## 🔬 Guided Inquiry 5: Visualizing and Analyzing Docking Poses

### Context

Docking scores alone are not sufficient - visual inspection is critical! Key things to check:

1. **Shape complementarity**: Does the ligand fit well in the pocket?
2. **Hydrogen bonds**: Are H-bond donors/acceptors satisfied?
3. **Hydrophobic contacts**: Are hydrophobic groups buried appropriately?
4. **Clashes**: Any steric clashes with protein atoms?
5. **Pose consistency**: Do top poses cluster in similar orientations?

### Your Task

Using your AI assistant, write code to:
1. Parse the docked poses from the PDBQT output
2. Visualize the best pose with the protein
3. Identify potential hydrogen bonds
4. Compare multiple poses to assess consistency

💡 **Prompting Tips**:
- Ask: "How do I parse multiple poses from a PDBQT file?"
- Use py3Dmol to show protein-ligand interactions
- Color-code poses to distinguish them

### Verification

After running your code, confirm:
- [ ] All poses visualized
- [ ] Ligand is positioned within the binding site
- [ ] No obvious steric clashes visible

📓 **Lab Notebook**: Describe the key protein-ligand interactions you observe.

In [ ]:
# Your code here



In [ ]:
# Your code here



### 📝 Solution Notes

**Pose clustering interpretation**:
- If top poses cluster together: High confidence in predicted binding mode
- If poses are scattered: Multiple binding modes possible, or difficult docking case

**What to look for visually**:
- Drug positioned in the expected pocket
- Polar groups pointing toward solvent or forming H-bonds
- Hydrophobic groups buried in nonpolar regions
- No significant steric overlaps

**Common binding motifs by target**:
- **DHFR**: H-bonds to Asp27, hydrophobic contacts with Ile5, Leu28
- **Kinases (ABL1, EGFR)**: H-bonds to hinge region, DFG motif interactions
- **AChE**: H-bonds in gorge, π-cation interactions with Trp

---

## 🔬 Guided Inquiry 6: Comparing to Experimental Structure

### Context

The ultimate validation of docking is comparing our predicted pose to the experimental (crystallographic) structure. We measure this using **RMSD** (Root Mean Square Deviation):

- RMSD < 2.0 Å: Excellent - considered a successful re-dock
- RMSD 2.0-4.0 Å: Fair - major interactions preserved but geometry differs
- RMSD > 4.0 Å: Poor - significantly different binding mode

This "re-docking" experiment tells us how well the docking method works for this target.

### Your Task

Using your AI assistant, write code to:
1. Extract atomic coordinates from the docked pose
2. Extract coordinates from the crystallographic ligand
3. Calculate RMSD between them (handling different atom orderings)
4. Visualize both poses overlaid

💡 **Prompting Tips**:
- Ask: "How do I calculate RMSD between two sets of atomic coordinates?"
- You may need to align molecules first for fair comparison
- Consider using heavy atoms only (excluding hydrogens)

### Verification

After running your code, confirm:
- [ ] RMSD calculated
- [ ] Both poses visualized for comparison
- [ ] RMSD interpretation provided

📓 **Lab Notebook**: Record the RMSD value and discuss whether the re-docking was successful.

In [ ]:
# Your code here



In [ ]:
# Your code here



### 📝 Solution Notes

**RMSD considerations**:
- We use heavy atoms only (no hydrogens) because H positions are often not resolved in X-ray
- Symmetric molecules may need special handling (e.g., phenyl ring rotations)
- RMSD is sensitive to atom mapping - incorrect mapping gives artificially high values

**What good RMSD means**:
- Vina correctly identified the binding mode
- Scoring function ranked the correct pose highly
- Method is appropriate for this target

**What poor RMSD might indicate**:
- Protein flexibility not captured (rigid protein limitation)
- Alternative binding mode is energetically similar
- Scoring function has limitations for this target
- The crystal structure may have artifacts (crystal packing effects)

**Next steps in real research**:
- Molecular dynamics to assess pose stability
- Multiple docking runs with different conditions
- Experimental validation (binding assays)

---

## 📊 Results Summary

Let's compile our docking results:

In [ ]:
# Your code here



---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Prepare protein and ligand structures for docking
- [ ] Define an appropriate binding box
- [ ] Run AutoDock Vina and interpret the output
- [ ] Visualize docking poses with py3Dmol
- [ ] Calculate RMSD to validate docking accuracy
- [ ] Explain limitations of physics-based docking

### Your lab notebook should include:

- [ ] Target information (PDB ID, protein name)
- [ ] Binding box parameters (center and size)
- [ ] Best docking score
- [ ] Re-docking RMSD value
- [ ] Description of observed protein-ligand interactions
- [ ] Assessment of docking quality

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Method Assessment**: How well did AutoDock Vina perform for your target? Would you trust these results for drug design decisions?

2. **Scoring Limitations**: The docking score is not a true binding energy. What factors does the scoring function miss? How might this affect virtual screening results?

3. **Comparison Ahead**: In the next talktorial, we'll use AI-powered docking methods (DiffDock, GNINA). What advantages might these have over physics-based methods like Vina?

4. **Protein Flexibility**: Vina treats the protein as rigid. For your target, how might protein flexibility affect drug binding? Are there known induced-fit effects?

---

## 📖 Further Reading

### Docking Theory & Practice
- [AutoDock Vina Documentation](https://autodock-vina.readthedocs.io/) - Official documentation
- [Molecular Docking Review](https://doi.org/10.1007/s12551-016-0247-1) - Pagadala et al., Biophysical Reviews 2017
- [Scoring Function Comparison](https://doi.org/10.1021/acs.jcim.0c01380) - Shen et al., JCIM 2021

### Tools Used
- [Meeko GitHub](https://github.com/forlilab/Meeko) - Ligand preparation tool
- [py3Dmol Documentation](https://3dmol.csb.pitt.edu/) - Molecular visualization

### Target-Specific Reading
- Find a docking study for your specific target in the literature
- Compare their methods and results to yours

---

## 🔗 Connection to Research

Molecular docking is a cornerstone of computational drug discovery:

**Virtual Screening**: Pharmaceutical companies dock millions of compounds against targets to find new leads. AutoDock Vina's speed (seconds per compound) makes this feasible.

**Lead Optimization**: Docking explains structure-activity relationships (SAR). Why does adding a methyl group improve potency? Docking can show the new hydrophobic contact.

**Resistance Prediction**: For your target, docking can predict whether mutations will affect drug binding - directly relevant to the resistance problem we're studying.

**Integration with AI**: In AI-PSCI-016, we'll see how AI methods like DiffDock approach docking differently - using diffusion models instead of physics-based scoring. The comparison will reveal when each approach excels.

---

*AI-PSCI-015 Complete. Proceed to AI-PSCI-016: AI-Powered Docking (DiffDock & GNINA).*